<a href="https://colab.research.google.com/github/nickhil-verma/MOSDAQ_SCRAPPING_SCRIPT/blob/main/Mosdaq_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install -q pymongo[srv] sentence-transformers requests python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 51.8 MB/s eta 0:00:00


In [ ]:
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
import requests
import numpy as np
from dotenv import load_dotenv
import os

# MongoDB + API
MONGO_URI =  os.getenv("MONGO_URI")
API_URL =  os.getenv("API_URL")

# Connect Mongo
client = MongoClient(MONGO_URI)
db = client["isro_Hackathon"]
collection = db["vectors"]
collection.drop()  # Clear old vectors

# Load embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_documents(query, k=3):
    query_embedding = model.encode([query])

    docs = list(collection.find({}, {"_id": 0}))
    texts = [doc["text"] for doc in docs]
    embeddings = [np.array(doc["embedding"]).reshape(1, -1) for doc in docs]
    embeddings = np.vstack(embeddings)

    similarities = cosine_similarity(query_embedding, embeddings)[0]
    top_k_indices = np.argsort(similarities)[::-1][:k]

    return [texts[i] for i in top_k_indices]


In [ ]:
def ask_ollama(query, context, model_name="gemma"):
    prompt = f"""You are an expert assistant. Answer the following question using ONLY the provided context.
If the answer is not in the context, say "I don't know."

Context:
{context}

Question:
{query}

Answer:"""

    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model_name,
                "prompt": prompt,
                "stream": False
            }
        )
        return response.json().get("response", "❌ No response from model.")
    except Exception as e:
        return f"❌ Error contacting Ollama: {e}"



In [ ]:
def rag_chatbot(question, model_name="gemma"):
    context_docs = get_top_k_documents(question)
    combined_context = "\n\n".join(context_docs)

    answer = ask_ollama(question, combined_context, model_name=model_name)

    print("🔍 Retrieved Context:\n", combined_context[:500], "\n")
    print("🧠 LLM Answer:\n", answer)


In [ ]:
rag_chatbot("What kind of data does MOSDAC LIVE URJA use?", model_name="gemma3")



🔍 Retrieved Context:
 MOSDAC Live – URJA. MOSDAC-LIVE is a web-enabled data and information visualization and analysis system of MOSDAC, SAC/ISRO. The system provides access to satellite data products and information products derived from satellite and model forecast in near real time basis. The website is currently a beta site under development and contains data layers from various organizations. Data layers include imagery from INSAT3S (including TIR1, TIR2, MIR, SWIR, Visible, Water Vapour), SOUNDER, and various m 

🧠 LLM Answer:
 ❌ Error contacting Ollama: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fbccdfc3e90>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [ ]:
def ask_ollama(query, context, model_name="gemma"):
    prompt = f"""You are a helpful assistant. Answer the following question using ONLY the provided context.
If the answer is not in the context, say "I don't know".

Context:
{context}

Question:
{query}

Answer:"""

    try:
        res = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": model_name,
                "prompt": prompt,
                "stream": False
            }
        )
        return res.json().get("response", "❌ No valid response.")
    except Exception as e:
        return f"❌ Error contacting Ollama: {e}"


In [ ]:
def rag_chatbot(question, model_name="gemma"):
    top_contexts = get_top_k_documents(question)
    combined_context = "\n\n".join(top_contexts)
    answer = ask_ollama(question, combined_context, model_name=model_name)

    print("🔍 Retrieved Context:\n", combined_context[:500], "\n")
    print("🧠 Answer:\n", answer)


In [ ]:
rag_chatbot("What kind of data does MOSDAC LIVE URJA use?", model_name="gemma3")


ValueError: Found array with dim 3. check_pairwise_arrays expected <= 2.